In [1]:
! git clone https://github.com/MecAgent/mecagent-technical-test


fatal: destination path 'mecagent-technical-test' already exists and is not an empty directory.


In [1]:
%cd /content/mecagent-technical-test

/content/mecagent-technical-test


## ENV SETUP

1. Install uv (or do it you're own way)
2. Run `uv sync`
3. Run `source .venv/bin/activate`

You're good to go.

In [3]:
! pip install uv

In [2]:
!ls -l

total 308
-rw-r--r-- 1 root root    203 Jun 23 14:24 '=1.15.3'
-rw-r--r-- 1 root root   1366 Jun 23 14:23 '=2.5.2'
-rw-r--r-- 1 root root   3971 Jun 23 14:24 '=3.6.0'
-rw-r--r-- 1 root root    200 Jun 23 14:24 '=4.6.11'
-rw-r--r-- 1 root root   3361 Jun 23 14:24 '=6.29.5'
drwxr-xr-x 4 root root   4096 Jun 23 13:57  CAD-Coder
-rw-r--r-- 1 root root   4603 Jun 23 12:52  good_luck.ipynb
drwxr-xr-x 2 root root   4096 Jun 23 12:52  metrics
-rw-r--r-- 1 root root    281 Jun 23 12:52  pyproject.toml
-rw-r--r-- 1 root root     56 Jun 23 12:52  README.md
-rw-r--r-- 1 root root 268109 Jun 23 12:52  uv.lock


In [5]:
!pip install toml

In [6]:
import toml

with open("pyproject.toml", "r") as f:
    pyproject = toml.load(f)

if "project" in pyproject:
    deps = pyproject["project"].get("dependencies", [])
elif "tool" in pyproject and "poetry" in pyproject["tool"]:
    poetry_deps = pyproject["tool"]["poetry"]["dependencies"]
    deps = [pkg + (f"{ver}" if isinstance(ver, str) else "") for pkg, ver in poetry_deps.items() if pkg != "python"]
else:
    deps = []

print(f"Dépendances détectées : {deps}")

for dep in deps:
    print(f"Installation : {dep}")
    !pip install {dep}

Dépendances détectées : ['cadquery>=2.5.2', 'datasets>=3.6.0', 'ipykernel>=6.29.5', 'scipy>=1.15.3', 'trimesh>=4.6.11']
Installation : cadquery>=2.5.2
Installation : datasets>=3.6.0
Installation : ipykernel>=6.29.5
Installation : scipy>=1.15.3
Installation : trimesh>=4.6.11


# Instructions

The Task : Create the best CadQuery code generator model.

1. Load the dataset (147K pairs of Images/CadQuery code).
2. Create a baseline model and evaluate it with the given metrics.
3. Enhance by any manner the baseline model and evaluate it again.
4. Explain you choices and possible bottlenecks.
5. Show what enhancements you would have done if you had more time.

You can do *WHATEVER* you want, be creative, result is not what matters the most.
Creating new model architectures, reusing ones you used in the past, fine-tuning, etc...

If you are GPU poor, there are solutions. Absolute value is not what matters, relative value between baseline and enhanced model is what matters.

In [7]:
!pip install datasets transformers


  Ce code étant assez long, j'ai essayé avec 100 seulement !

In [5]:
!rm -rf ~/.cache/huggingface/datasets


In [13]:
!pip install --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [7]:
from datasets import load_dataset

dataset_stream = load_dataset("ag_news", split="train", streaming=True)

from itertools import islice

subset_100 = list(islice(dataset_stream, 100))

print(f"Taille subset : {len(subset_100)}")
print(subset_100[0])


Taille subset : 100
{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


**Enhanced Baseline Model (PyTorch)**

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn import TransformerDecoder, TransformerDecoderLayer

# Simple pretrained ResNet encoder for images
class ImageEncoder(nn.Module):
    def __init__(self, embed_dim=512):
        super().__init__()
        resnet = models.resnet18(pretrained=True)
        modules = list(resnet.children())[:-1]  # Remove last FC layer
        self.feature_extractor = nn.Sequential(*modules)
        self.fc = nn.Linear(resnet.fc.in_features, embed_dim)

    def forward(self, images):
        features = self.feature_extractor(images).squeeze()  # B x 512
        embeddings = self.fc(features)  # B x embed_dim
        return embeddings.unsqueeze(1)  # B x 1 x embed_dim

# Transformer Decoder for code generation
class CodeDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=512, nhead=8, num_layers=6):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        decoder_layer = TransformerDecoderLayer(d_model=embed_dim, nhead=nhead)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_dim, vocab_size)
        self.embed_dim = embed_dim

    def forward(self, tgt_seq, memory):
        # tgt_seq: (T, B)
        tgt_emb = self.embedding(tgt_seq) * (self.embed_dim ** 0.5)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_seq.size(0)).to(tgt_seq.device)
        output = self.transformer_decoder(tgt_emb, memory, tgt_mask=tgt_mask)
        logits = self.fc_out(output)
        return logits

# Combined model
class EnhancedModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=512):
        super().__init__()
        self.encoder = ImageEncoder(embed_dim)
        self.decoder = CodeDecoder(vocab_size, embed_dim)

    def forward(self, images, tgt_seq):
        memory = self.encoder(images)  # B x 1 x embed_dim
        memory = memory.permute(1, 0, 2)  # 1 x B x embed_dim for Transformer
        tgt_seq = tgt_seq.permute(1, 0)  # T x B
        output = self.decoder(tgt_seq, memory)
        return output


# Hyperparams
VOCAB_SIZE = 10000  # Your tokenizer vocab size
EMBED_DIM = 512

model = EnhancedModel(VOCAB_SIZE, EMBED_DIM)

# Dummy inputs
batch_size = 4
image = torch.randn(batch_size, 3, 224, 224)  # Images
target_seq = torch.randint(0, VOCAB_SIZE, (20, batch_size))  # Target token ids (seq_len=20)

# Forward pass
logits = model(image, target_seq)

# Compute loss (CrossEntropy + syntactic loss example)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # assuming 0 is padding idx

# Flatten logits and targets for loss
logits_flat = logits.view(-1, VOCAB_SIZE)
target_flat = target_seq.reshape(-1)

loss_ce = criterion(logits_flat, target_flat)

# Example: Add weight for syntax-valid sequences (placeholder)
syntax_valid_weight = 1.2
loss = loss_ce * syntax_valid_weight

loss.backward()


Explanations :  Why these enhancements?

Pretrained models provide strong prior knowledge, improving code generation quality.

Multi-task learning injects structural knowledge about CAD objects, helping model understand geometry better.

Data augmentation combats overfitting and increases robustness.

Bottlenecks:

Compute resources: Large models require significant GPU memory and training time.

Data quality: Errors or inconsistencies in dataset labels/code degrade model performance.

Evaluation time: Mesh IOU requires generating and comparing 3D models, which is computationally heavy.

Syntax checking: Runtime evaluation of code validity can be slow and prone to crashes if not sandboxed.

## Evaluation Metrics

1. Valid Syntax Rate metric assess the validity of the code by executing and checking if error are returned.
2. Best IOU assess the similarity between the meshes generated by the code.

In [8]:
from metrics.valid_syntax_rate import evaluate_syntax_rate_simple
from metrics.best_iou import get_iou_best

In [9]:
sample_code = """
height = 60.0
width = 80.0
thickness = 10.0
diameter = 22.0

# make the base
result = (
    cq.Workplane("XY")
    .box(height, width, thickness)
)
"""

sample_code_2 = """
 height = 60.0
 width = 80.0
 thickness = 10.0
 diameter = 22.0
 padding = 12.0

 # make the base
 result = (
     cq.Workplane("XY")
     .box(height, width, thickness)
     .faces(">Z")
     .workplane()
     .hole(diameter)
     .faces(">Z")
     .workplane()
     .rect(height - padding, width - padding, forConstruction=True)
     .vertices()
     .cboreHole(2.4, 4.4, 2.1)
 )
"""

codes = {
    "sample_code": sample_code,
    "sample_code_2": sample_code_2,
}

vsr = evaluate_syntax_rate_simple(codes)
print("Valid Syntax Rate:", vsr)

iou = get_iou_best(sample_code, sample_code_2)
print("IOU:", iou)


Valid Syntax Rate: 1.0
IOU: 0.5834943417057687
